<a href="https://colab.research.google.com/github/demyanchuk-nestor/AI_for_Medical_Diagnosis/blob/master/Week-1/UKR_Chest_X_Ray_Medical_Diagnosis_with_Deep_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/demyanchuk-nestor/AI_for_Medical_Diagnosis
%cd 'AI_for_Medical_Diagnosis/Week-1'

# Chest X-Ray Medical Diagnosis with Deep Learning

<img src="https://github.com/hardik0/AI-for-Medicine-Specialization/blob/master/AI-for-Medical-Diagnosis/Week-1/xray-header-image.png?raw=true" style="padding-top: 50px;width: 87%;left: 0px;margin-left: 0px;margin-right: 0px;">
** Завдання 7, 8!**

У цьому завданні! Ви вивчите діагностику медичних зображень шляхом створення сучасного класифікатора рентгенівських знімків грудної клітки за допомогою Keras.

У завданні будуть розглянуті деякі етапи побудови та оцінки цієї моделі класифікатора з глибоким навчанням. Зокрема, ви повинні будете
- Попередньо обробити та підготувати реальний набір рентгенівських знімків
- Використовувати навчання з перенавчанням для перенавчання моделі DenseNet для класифікації рентгенівських зображень
- Вивчите техніку обробки дисбалансу класів
- Вимірювання діагностичної ефективності шляхом обчислення AUC (площа під кривою) для кривої ROC (робоча характеристика приймача)
- Візуалізувати активність моделі за допомогою GradCAM

Виконуючи це завдання, ви дізнаєтесь про наступні теми:

- Підготовка даних
  - Візуалізація даних
  - Запобігання витоку даних
- Розробка моделі
  - Усунення дисбалансу класів
  - Використання попередньо навчених моделей з використанням навчання з переносом
- Оцінювання
  - Криві AUC та ROC





Використовуйте ці посилання, щоб переходити до певних розділів цього завдання!

- [1. Імпорт даних і функцій](#1)
- [2. Завантажте набори даних](#2)
- [3. Розробка моделі](#3)
- [4. Навчання [необов’язково]](#4)

- [5. Прогнозування та оцінка](#5)



<a name='1'></a>
##  1. Дані та функції для імпорту¶

Ми скористаємося наступними даними:
- `*numpy*` та `*pandas*` для маніпулювання даними
- `*matplotlib.pyplot*` та `*seaborn*` для створення графіків для візуалізації
- `*util*` надасть локально визначені утиліти, які були надані для цього завдання



Запустіть наступну комірку, щоб імпортувати всі необхідні пакунки.


In [ ]:
!pip install numpy==1.24.3
!pip install pandas==1.5.2
!pip install tensorflow==2.13.1

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
# import tensorflow.python.keras.backend as K
import tensorflow.python.keras.backend as K
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
from tensorflow.keras.models import load_model

import util
from itertools import compress

<a name='2'></a>

**2 Завантаження наборів даних**

Для цього завдання ми будемо використовувати набір даних ChestX-ray8, який містить 108 948 рентгенівських знімків у прямій проекції 32 717 унікальних пацієнтів.

Кожне зображення в наборі даних містить кілька текстових міток, що ідентифікують 14 різних патологічних станів.
Вони, в свою чергу, можуть бути використані лікарями для діагностики 8 різних захворювань.
Ми використаємо ці дані для розробки єдиної моделі, яка надасть бінарні класифікаційні прогнози для кожної з 14 позначених патологій.
Іншими словами, вона передбачатиме «позитивний» або «негативний» результат для кожної з патологій.
Ви можете безкоштовно завантажити весь набір даних тут.


Щоб полегшити вам роботу, ми обробили мітки для нашої невеликої вибірки і згенерували три нові файли, з яких ви можете почати. Ось ці файли:




 Прочитайте дані
Відкриємо ці файли за допомогою бібліотеки [pandas](https://pandas.pydata.org/)


In [ ]:
train_df = pd.read_csv("../nih/train-small.csv")
valid_df = pd.read_csv("../nih/valid-small.csv")
test_df = pd.read_csv("../nih/test.csv")

train_df.head()

In [ ]:
labels = ['Cardiomegaly',
          'Emphysema',
          'Effusion',
          'Hernia',
          'Infiltration',
          'Mass',
          'Nodule',
          'Atelectasis',
          'Pneumothorax',
          'Pleural_Thickening',
          'Pneumonia',
          'Fibrosis',
          'Edema',
          'Consolidation']


Візуалізація зображень за класами


In [ ]:
# Visualize images for class "Atelectasis"
train_class = train_df.loc[train_df['Atelectasis']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Atelectasis" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Cardiomegaly"
train_class = train_df.loc[train_df['Cardiomegaly']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Cardiomegaly" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Consolidation"
train_class = train_df.loc[train_df['Consolidation']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Consolidation" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Edema"
train_class = train_df.loc[train_df['Edema']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Edema" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Effusion"
train_class = train_df.loc[train_df['Effusion']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Effusion" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Emphysema"
train_class = train_df.loc[train_df['Emphysema']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Emphysema" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Fibrosis"
train_class = train_df.loc[train_df['Fibrosis']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Fibrosis" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Hernia"
train_class = train_df.loc[train_df['Hernia']==1,:]

number_of_images = 2
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Hernia" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(1, 2, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Infiltration"
train_class = train_df.loc[train_df['Infiltration']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Infiltration" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Mass"
train_class = train_df.loc[train_df['Mass']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Mass" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

In [ ]:
# Visualize images for class "Nodule"
train_class = train_df.loc[train_df['Nodule']==1,:]

number_of_images = 9
custom_df = train_class.sample(n=number_of_images)

# Plot a processed image
sns.set_style("white")
fig = plt.figure(figsize=(20,10))

fig.suptitle('Images with "Nodule" diagnosis', fontsize=16)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()

<a name='Ex-1'></a>
**Вправа 1** - Перевірка витоку даних
У комірці нижче напишіть функцію для перевірки наявності витоку між двома наборами даних. Ми використаємо її, щоб переконатися, що в тестовому наборі немає пацієнтів, які також присутні в тренувальному або перевірочному наборах.

In [ ]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def check_for_leakage(df1, df2, patient_col):
    """
    Return True if there any patients are in both df1 and df2.

    Args:
        df1 (dataframe): dataframe describing first dataset
        df2 (dataframe): dataframe describing second dataset
        patient_col (str): string name of column with patient IDs

    Returns:
        leakage (bool): True if there is leakage, otherwise False
    """

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    df1_patients_unique = set(df1[patient_col])
    df2_patients_unique = set(df2[patient_col])

    patients_in_both_groups = len(list(df1_patients_unique.intersection(df2_patients_unique)))

    # leakage contains true if there is patient overlap, otherwise false.
    leakage = False if patients_in_both_groups == 0 else True # boolean (true if there is at least 1 patient in both groups)

    ### END CODE HERE ###

    return leakage

In [ ]:
# test
print("test case 1")
df1 = pd.DataFrame({'patient_id': [0, 1, 2]})
df2 = pd.DataFrame({'patient_id': [2, 3, 4]})
print("df1")
print(df1)
print("df2")
print(df2)
print(f"leakage output: {check_for_leakage(df1, df2, 'patient_id')}")
print("-------------------------------------")
print("test case 2")
df1 = pd.DataFrame({'patient_id': [0, 1, 2]})
df2 = pd.DataFrame({'patient_id': [3, 4, 5]})
print("df1:")
print(df1)
print("df2:")
print(df2)

print(f"leakage output: {check_for_leakage(df1, df2, 'patient_id')}")

##### Очікувані результати

```Python
test case 1
df1
   patient_id
0           0
1           1
2           2
df2
   patient_id
0           2
1           3
2           4
leakage output: True
-------------------------------------
test case 2
df1:
   patient_id
0           0
1           1
2           2
df2:
   patient_id
0           3
1           4
2           5
leakage output: False
```

Перейдіть до наступної клітинки, щоб перевірити, чи є пацієнти в тренуванні та тестуванні, або в дійсності та тестуванні.

In [ ]:
print("leakage between train and test: {}".format(check_for_leakage(train_df, test_df, 'PatientId')))
print("leakage between valid and test: {}".format(check_for_leakage(valid_df, test_df, 'PatientId')))

Якщо ми отримали `***False***` для обох, то ми готові почати підготовку наборів даних для навчання. Не забувайте завжди перевіряти на витік даних!

Для цього ми скористаємося готовим класом *ImageDataGenerator *з фреймворку *Keras*, який дозволяє створити «генератор» зображень, вказаних у фреймі даних.
Цей клас також забезпечує підтримку базового доповнення даних, такого як випадкове горизонтальне перевертання зображень.
Ми також використовуємо генератор для перетворення значень у кожній партії так, щоб їхнє середнє значення дорівнювало 0, а стандартне відхилення - 1.
Це полегшить навчання моделі, стандартизувавши розподіл вхідних даних.
Генератор також перетворює наші одноканальні рентгенівські зображення (у сірій шкалі) у триканальний формат, повторюючи значення на зображенні у всіх каналах.
Нам це потрібно, оскільки попередньо навчена модель, яку ми будемо використовувати, вимагає триканальних входів.
Оскільки це переважно питання читання і розуміння документації Keras, ми реалізували генератор для вас. Є кілька речей, на які слід звернути увагу:

Ми нормалізуємо середнє та стандартне відхилення даних
Ми перемішуємо вхідні дані після кожної епохи.
Ми встановлюємо розмір зображення 320px на 320px

In [ ]:
print("getting train generator...")

# Normalize images
image_generator = ImageDataGenerator(
    samplewise_center=True, #Set each sample mean to 0.
    samplewise_std_normalization= True # Divide each input by its standard deviation
)
# Flow from directory with specified batch size and target image size
train_generator = image_generator.flow_from_dataframe(
        dataframe=train_df,
        directory="../nih/images_small/",
        x_col="Image", # features
        y_col= labels, # labels
        class_mode="raw", # 'Mass' column should be in train_df
        batch_size= 8, # images per batch
        seed= 1,
        shuffle=True, # shuffle the rows or not
        target_size=(320,320) # width and height of output image
)

####
Створіть окремий генератор для валідних і тестових наборів
Тепер нам потрібно створити новий генератор для валідних і тестових даних.

Чому ми не можемо використовувати той самий генератор, що і для навчальних даних?

Подивіться на генератор, який ми написали для навчальних даних.

Він нормалізує кожне зображення для кожної партії, тобто використовує пакетну статистику.
Ми не повинні робити цього з тестовими та валідаційними даними, оскільки в реальному житті ми не обробляємо вхідні зображення пакетом за раз (ми обробляємо по одному зображенню за раз).
Знання середнього значення за партію тестових даних фактично дасть нашій моделі перевагу.
Модель не повинна мати жодної інформації про тестові дані.
Що нам потрібно зробити, так це нормалізувати вхідні тестові дані, використовуючи статистику, обчислену на навчальній вибірці.

Ми реалізуємо це у функції нижче.
Є одне технічне зауваження. В ідеалі, ми хотіли б обчислити середнє значення вибірки і стандартне відхилення, використовуючи весь навчальний набір.
Однак, оскільки він надзвичайно великий, це забрало б багато часу.
Для економії часу ми візьмемо випадкову вибірку з набору даних і обчислимо вибіркове середнє і вибіркове стандартне відхилення.
Маючи готову функцію генератора, створимо один генератор для наших навчальних даних і по одному для тестових і валідаційних наборів даних.


Маючи готову функцію генератора, давайте створимо один генератор для наших навчальних даних і по одному для тестових і валідаційних наборів даних.


In [ ]:
print("getting test and valid generators...")

# Flow from directory with specified batch size and target image size
print("raw_train_generator")
raw_train_generator = image_generator.flow_from_dataframe(
        dataframe=train_df,
        directory="../nih/images_small/",
        x_col="Image", # features
        y_col= labels, # labels
        class_mode="raw", # 'Mass' column should be in train_df
        batch_size= 100, # images per batch
        shuffle=True, # shuffle the rows or not
        target_size=(320,320) # width and height of output image
)

# get data sample
batch = next(raw_train_generator)
data_sample = batch[0]

# use sample to fit mean and std for test set generator
image_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization= True)

# fit generator to sample from training data
image_generator.fit(data_sample)

# get valid generator
print("valid_generator")
valid_generator = image_generator.flow_from_dataframe(
        dataframe=valid_df,
        directory="../nih/images_small/",
        x_col="Image", # features
        y_col= labels, # labels
        class_mode="raw", # 'Mass' column should be in valid_df
        batch_size= 8, # images per batch
        seed= 1,
        shuffle=False, # shuffle the rows or not
        target_size=(320,320) # width and height of output image
)

# get test generator
print("test_generator")
test_generator = image_generator.flow_from_dataframe(
        dataframe=test_df,
        directory="../nih/images_small/",
        x_col="Image", # features
        y_col= labels, # labels
        class_mode="raw", # 'Mass' column should be in test_df
        batch_size= 8, # images per batch
        seed= 1,
        shuffle=False, # shuffle the rows or not
        target_size=(320,320) # width and height of output image
)

Давайте подивимось, що генератор дає нашій моделі під час навчання та перевірки. Це можна зробити за допомогою виклику функції __get_item__(index):


In [ ]:
# Plot a processed image
sns.set_style("white")
plt.figure(figsize=(20,10))

for i in range(6):
    plt.subplot(3, 3, i + 1)
    generated_image, label = train_generator.__getitem__(i)
    plt.imshow(generated_image[i])
    plt.colorbar()
#plt.title('Raw Chest X Ray Image')
#print(f"The dimensions of the image are {generated_image.shape[1]} pixels width and {generated_image.shape[2]} pixels height")
#print(f"The maximum pixel value is {generated_image.max():.4f} and the minimum is {generated_image.min():.4f}")
#print(f"The mean value of the pixels is {generated_image.mean():.4f} and the standard deviation is {generated_image.std():.4f}")

<a name='3'></a>
**3 Розробка моделі**

Тепер ми перейдемо до навчання та розробки моделі. Однак перед тим, як навчати нейронну мережу, нам потрібно вирішити кілька практичних проблем. Перша - це дисбаланс класів.


<a name='3-1'>

Вирішення проблеми дисбалансу класів
Однією з проблем при роботі з наборами медичних діагностичних даних є значний дисбаланс класів, присутній у таких наборах даних. Давайте побудуємо графік частоти кожної з міток у нашому наборі даних:

In [ ]:
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_generator.labels, axis=0))
plt.title("Frequency of Each Class")
plt.show()

З цього графіка ми бачимо, що поширеність позитивних випадків суттєво відрізняється для різних патологій. (Ці тенденції також відображають тенденції в повному наборі даних).

Патологія грижі має найбільший дисбаланс: частка позитивних випадків лікування становить близько 0,2%.
Але навіть патологія Інфільтрація, яка має найменший дисбаланс, має лише 17,5% випадків навчання, позначених як позитивні.
В ідеалі, ми б навчали нашу модель, використовуючи рівномірно збалансований набір даних, щоб позитивні і негативні навчальні випадки вносили однаковий внесок у втрати.

Якщо ми використовуємо нормальну функцію втрат перехресної ентропії з дуже незбалансованим набором даних, як ми бачимо тут, то алгоритм буде зацікавлений надати пріоритет класу більшості (тобто негативному в нашому випадку), оскільки він вносить більший внесок у втрати.


<a name='Ex-2'></a>
**Вправа 2**
Обчислення частот класів
Заповніть функцію нижче, щоб обчислити ці частоти для кожної мітки в нашому наборі даних.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Use numpy.sum(a, axis=), and choose the axis (0 or 1) </li>
</ul>
</p>


In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)

def compute_class_freqs(labels):
    """
    Compute positive and negative frequences for each class.

    Args:
        labels (np.array): matrix of labels, size (num_examples, num_classes)
    Returns:
        positive_frequencies (np.array): array of positive frequences for each
                                         class, size (num_classes)
        negative_frequencies (np.array): array of negative frequences for each
                                         class, size (num_classes)
    """
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # total number of patients (rows)
    N = labels.shape[0]

    positive_frequencies = np.sum(labels,0)/N
    negative_frequencies = np.ones_like(positive_frequencies) - positive_frequencies
    ### END CODE HERE ###
    return positive_frequencies, negative_frequencies

In [ ]:
# Test
labels_matrix = np.array(
    [[1, 0, 0],
     [0, 1, 1],
     [1, 0, 1],
     [1, 1, 1],
     [1, 0, 1]]
)
print("labels:")
print(labels_matrix)

test_pos_freqs, test_neg_freqs = compute_class_freqs(labels_matrix)

print(f"pos freqs: {test_pos_freqs}")

print(f"neg freqs: {test_neg_freqs}")

Очікувані результати

```Python
labels:
[[1 0 0]
 [0 1 1]
 [1 0 1]
 [1 1 1]
 [1 0 1]]
pos freqs: [0.8 0.4 0.8]
neg freqs: [0.2 0.6 0.2]
```

Тепер ми обчислимо частоти для наших навчальних даних.


In [ ]:
freq_pos, freq_neg = compute_class_freqs(train_generator.labels)
freq_pos

Давайте візуалізуємо ці два співвідношення внесків поруч для кожної з патологій:


In [ ]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": freq_pos})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} for l,v in enumerate(freq_neg)], ignore_index=True)
plt.xticks(rotation=90)
f = sns.barplot(x="Class", y="Value", hue="Label" ,data=data)

In [ ]:
pos_weights = freq_neg
neg_weights = freq_pos
pos_contribution = np.array(freq_pos) * pos_weights
neg_contribution = np.array(freq_neg) * neg_weights

Давайте перевіримо це, знову зобразивши два внески поруч один з одним на графіку:

In [ ]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": pos_contribution})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v}
                        for l,v in enumerate(neg_contribution)], ignore_index=True)
plt.xticks(rotation=90)
sns.barplot(x="Class", y="Value", hue="Label" ,data=data);

<a name='Ex-3'></a>
Вправа 3
Зважені втрати
Заповніть функцію *weighted_loss* нижче, щоб повернути функцію втрат, яка обчислює зважені втрати для кожної партії. Пам'ятайте, що для багатокласових втрат ми додаємо середні втрати для кожного окремого класу. Зауважте, що ми також хочемо додати невелике значення, ϵ , до прогнозованих значень перед тим, як брати їхні логарифми. Це робиться для того, щоб уникнути числової помилки, яка може виникнути, якщо прогнозоване значення виявиться нульовим.

Примітка
Будь ласка, використовуйте функції *Keras* для обчислення середнього значення та логарифму.


- [Keras.mean](https://www.tensorflow.org/api_docs/python/tf/keras/backend/mean)
- [Keras.log](https://www.tensorflow.org/api_docs/python/tf/keras/backend/log)



In [ ]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    """
    Return weighted loss function given negative weights and positive weights.

    Args:
      pos_weights (np.array): array of positive weights for each class, size (num_classes)
      neg_weights (np.array): array of negative weights for each class, size (num_classes)

    Returns:
      weighted_loss (function): weighted loss function
    """
    def weighted_loss(y_true, y_pred):
        """
        Return weighted loss value.

        Args:
            y_true (Tensor): Tensor of true labels, size is (num_examples, num_classes)
            y_pred (Tensor): Tensor of predicted labels, size is (num_examples, num_classes)
        Returns:
            loss (Tensor): overall scalar loss summed across all classes
        """
        # initialize loss to zero
        loss = 0.0

        ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

        for i in range(len(pos_weights)):
            # for each class, add average weighted loss for that class
            positive_term_loss = pos_weights[i]*y_true[:,i]*K.log(y_pred[:,i] + epsilon)
            negative_term_loss = neg_weights[i]*(1-y_true[:,i])*K.log(1-y_pred[:,i] + epsilon)
            loss +=  -K.mean(positive_term_loss + negative_term_loss)  #complete this line
        return loss

        ### END CODE HERE ###
    return weighted_loss

Тепер давайте протестуємо нашу функцію на деяких простих прикладах.

In [ ]:
# Test
sess = K.get_session()
with sess.as_default() as sess:
    print("Test example:\n")
    y_true = K.constant(np.array(
        [[1, 1, 1],
         [1, 1, 0],
         [0, 1, 0],
         [1, 0, 1]]
    ))
    print("y_true:\n")
    print(y_true.eval())

    w_p = np.array([0.25, 0.25, 0.5])
    w_n = np.array([0.75, 0.75, 0.5])
    print("\nw_p:\n")
    print(w_p)

    print("\nw_n:\n")
    print(w_n)

    y_pred_1 = K.constant(0.7*np.ones(y_true.shape))
    print("\ny_pred_1:\n")
    print(y_pred_1.eval())

    y_pred_2 = K.constant(0.3*np.ones(y_true.shape))
    print("\ny_pred_2:\n")
    print(y_pred_2.eval())

    # test with a large epsilon in order to catch errors
    L = get_weighted_loss(w_p, w_n, epsilon=1)

    print("\nIf we weighted them correctly, we expect the two losses to be the same.")
    L1 = L(y_true, y_pred_1).eval()
    L2 = L(y_true, y_pred_2).eval()
    print(f"\nL(y_pred_1)= {L1:.4f}, L(y_pred_2)= {L2:.4f}")
    print(f"Difference is L1 - L2 = {L1 - L2:.4f}")

<a name='3-3'></a>
Далі ми використаємо попередньо навчену модель DenseNet121 (https://www.kaggle.com/pytorch/densenet121), яку ми можемо завантажити безпосередньо з Keras, а потім додати два шари поверх неї:

Шар GlobalAveragePooling2D для отримання середнього значення останніх шарів згортки з DenseNet121.
Шар Dense з сигмоїдною активацією для отримання логів прогнозу для кожного з наших класів.
Ми можемо задати власну функцію втрат для моделі, вказавши параметр loss у функції compile.



In [ ]:
# create the base pre-trained model
base_model = DenseNet121(weights='../nih/densenet.hdf5', include_top=False)

x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)

# and a logistic layer
predictions = Dense(len(labels), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss=get_weighted_loss(pos_weights, neg_weights))

<a name='4'></a>

**4 Навчання**

Коли наша модель готова до навчання, ми використаємо функцію model.fit() у Keras для навчання нашої моделі.

Ми тренуємось на невеликій підмножині набору даних (~1%).
Тому на даному етапі нам важливо переконатися, що втрати на навчальній вибірці зменшуються.
Оскільки навчання може зайняти значний час, у педагогічних цілях ми вирішили не навчати модель тут, а завантажити набір попередньо навчених ваг у наступному розділі. Однак ви можете використовувати код, показаний нижче, щоб потренувати модель локально на вашому комп'ютері або в Colab.

ПРИМІТКА: Не запускайте наведений нижче код на платформі Coursera, оскільки він перевищить обмеження пам'яті платформи.

Код на Python для навчання моделі:

```python
history = model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              steps_per_epoch=100,
                              validation_steps=25,
                              epochs = 3)

plt.plot(history.history['loss'])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Training Loss Curve")
plt.show()
```

<a name='4-1'></a>
Навчання на великому наборі даних
Враховуючи, що оригінальний набір даних має розмір 40 ГБ+, а процес навчання на повному наборі даних займає кілька годин, ми навчили модель на машині з графічним процесором і надали вам файл ваг нашої моделі (з розміром партії 32) для використання в подальшому завданні.

Архітектура моделі для нашої попередньо навченої моделі точно така ж, але ми використали кілька корисних «зворотних викликів» Keras для цього навчання. Витратьте час, щоб прочитати про ці зворотні виклики на дозвіллі, оскільки вони будуть дуже корисними для управління довготривалими навчальними сесіями:

Ви можете використовувати ModelCheckpoint для моніторингу метрики val_loss вашої моделі та збереження знімка вашої моделі в точці.
Ви можете використовувати утиліту TensorBoard для моніторингу ваших тренувань в реальному часі за допомогою тензорної дошки Tensorflow.
Ви можете використовувати ReduceLROnPlateau для повільного зниження швидкості навчання вашої моделі, коли вона перестає покращуватися за такою метрикою, як val_loss, щоб точно налаштувати модель на останніх етапах навчання.
Ви можете використовувати зворотний виклик EarlyStopping, щоб зупинити завдання навчання, коли ваша модель перестане покращуватися за показником втрати валідації. Ви можете встановити значення терпіння - кількість епох, після яких модель не покращується, після чого навчання буде припинено. Цей колбек також може зручно відновити ваги для найкращої метрики в кінці навчання для вашої моделі.
Ви можете прочитати про ці та інші корисні зворотні виклики Keras тут (https://keras.io/callbacks/).



In [ ]:
model.load_weights("../nih/pretrained_model.h5")

<a name='5'></a>
**5 Прогнозування та оцінка**


Тепер, коли у нас є модель, давайте оцінимо її на нашому тестовому наборі. Ми можемо зручно використовувати функцію predict_generator для генерації прогнозів для зображень у нашому тестовому наборі.

Примітка: наступна комірка може працювати близько 4 хвилин.


In [ ]:
predicted_vals = model.predict(test_generator, steps = len(test_generator))

<a name='5-1'></a>

Крива ROC та AUROC
Ми розглянемо тему оцінювання моделей набагато детальніше в наступних розділах, а зараз ми розглянемо обчислення метрики, яка називається AUC (площа під кривою) з кривої ROC (робоча характеристика приймача). Цей показник також називають значенням AUROC, але ви побачите, що всі три терміни відносяться до цієї методики і часто використовуються майже як взаємозамінні.

Наразі, щоб інтерпретувати графік, вам потрібно знати, що крива, яка знаходиться лівіше і вище, має більшу «площу» під собою, і вказує на те, що модель працює краще.

Ми будемо використовувати функцію util.get_roc_curve(), яка була надана вам у файлі util.py. Перегляньте цю функцію і зверніть увагу на використання функцій бібліотеки sklearn для генерації ROC-кривих і значень AUROC для нашої моделі.
 ([Receiver Operating Characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic))

- [roc_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)
- [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

In [ ]:
auc_rocs = util.get_roc_curve(labels, predicted_vals, test_generator)

You can compare the performance to the AUCs reported in the original ChexNeXt paper in the table below:

Цей метод також використовує кілька інших прийомів, таких як самонавчання та ансамблі, які можуть дати значний поштовх для покращення виконання:

<img src="https://journals.plos.org/plosmedicine/article/figure/image?size=large&id=10.1371/journal.pmed.1002686.t001" width="80%">




Для отримання більш детальної інформації про найкращі методи та їхню ефективність на цьому наборі даних, ми рекомендуємо вам ознайомитися з наступними статтями:
- [CheXNet](https://arxiv.org/abs/1711.05225)
- [CheXpert](https://arxiv.org/pdf/1901.07031.pdf)
- [ChexNeXt](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1002686)

Однією з проблем використання глибокого навчання в медицині є те, що складна архітектура нейронних мереж робить їх набагато складнішими для інтерпретації порівняно з традиційними моделями машинного навчання (наприклад, лінійними моделями).

Одним з найпоширеніших підходів, спрямованих на підвищення інтерпретованості моделей для задач комп'ютерного зору, є використання карт активації класів (Class Activation Maps, CAM).

Карти активації класів корисні для розуміння того, куди «дивиться» модель при класифікації зображення.
У цьому розділі ми використаємо техніку GradCAM (https://arxiv.org/abs/1610.02391)для створення теплової карти, що виділяє важливі області на зображенні для прогнозування патологічного стану.

Це робиться шляхом вилучення градієнтів кожного передбачуваного класу, що перетікають у фінальний згорточний шар нашої моделі. Подивіться на util.compute_gradcam, який ми надали вам у файлі util.py, щоб побачити, як це робиться у фреймворку Keras.
Варто зазначити, що GradCAM не надає повного пояснення причин кожної ймовірності класифікації.

Однак, це все ще корисний інструмент для «налагодження» нашої моделі та покращення прогнозу, щоб експерт міг підтвердити, що прогноз дійсно обумовлений тим, що модель фокусується на правильних ділянках зображення.


Спочатку ми завантажимо невелику навчальну вибірку і налаштуємо її на перегляд 4 класів з найвищими показниками AUC.

In [ ]:
df = pd.read_csv("../nih/train-small.csv")
IMAGE_DIR = "../nih/images_small/"

# only show the lables with top 4 AUC
labels_to_show = np.take(labels, np.argsort(auc_rocs)[::-1])[:4]

Тепер давайте подивимося на кілька конкретних зображень.

In [ ]:
util.compute_gradcam(model, '00008270_015.png', IMAGE_DIR, df, labels, labels_to_show)

In [ ]:
util.compute_gradcam(model, '00011355_002.png', IMAGE_DIR, df, labels, labels_to_show)

In [ ]:
util.compute_gradcam(model, '00029855_001.png', IMAGE_DIR, df, labels, labels_to_show)

In [ ]:
util.compute_gradcam(model, '00005410_000.png', IMAGE_DIR, df, labels, labels_to_show)

Вітаємо, ви виконали перше завдання першого курсу! Ви навчилися попередньо обробляти дані, перевіряти їх на витік, навчати попередньо навчену модель та оцінювати за допомогою AUC. Чудова робота!

Отримання прогнозів моделі на випадково вибраних зображеннях з тестового набору даних


In [ ]:
# Visualize images for true class "Atelectasis"
class_label = 'Atelectasis'
test_class = test_df.loc[test_df[class_label]==1,:]

# Define number of images to plot
number_of_images = 9
custom_df = test_class.sample(n=number_of_images)

# Get predicted labels
labels = custom_df.columns.tolist()[2:]
true_classes = custom_df.iloc[:,2:].values==1
selected_predictions = predicted_vals[custom_df.index,:]>0.5

true_labels, pred_labels = [], []
for i in np.arange(0, selected_predictions.shape[0]):
    true_labels.append(list(compress(labels, true_classes[i,:])))
    pred_labels.append(list(compress(labels, selected_predictions[i,:])))

# Plot a processed image
sns.set_style("white")
plt.figure(figsize=(20,10))
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=1.5,
                    wspace=0.4,
                    hspace=0.4)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()
    plt.title('True class: ' + '\n' +  ', '.join(true_labels[i]) + \
              '\n' 'Predicted class: ' + '\n' + ', '.join(pred_labels[i]) + '\n', fontsize=16)

In [ ]:
# Visualize images for true class "Emphysema"
class_label = 'Emphysema'
test_class = test_df.loc[test_df[class_label]==1,:]

# Define number of images to plot
number_of_images = 9
custom_df = test_class.sample(n=number_of_images)

# Get predicted labels
labels = custom_df.columns.tolist()[2:]
true_classes = custom_df.iloc[:,2:].values==1
selected_predictions = predicted_vals[custom_df.index,:]>0.5

true_labels, pred_labels = [], []
for i in np.arange(0, selected_predictions.shape[0]):
    true_labels.append(list(compress(labels, true_classes[i,:])))
    pred_labels.append(list(compress(labels, selected_predictions[i,:])))

# Plot a processed image
sns.set_style("white")
plt.figure(figsize=(20,10))
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=1.5,
                    wspace=0.4,
                    hspace=0.4)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()
    plt.title('True class: ' + '\n' +  ', '.join(true_labels[i]) + \
              '\n' 'Predicted class: ' + '\n' + ', '.join(pred_labels[i]) + '\n', fontsize=16)

In [ ]:
# Visualize images for true class "Infiltration"
class_label = 'Infiltration'
test_class = test_df.loc[test_df[class_label]==1,:]

# Define number of images to plot
number_of_images = 9
custom_df = test_class.sample(n=number_of_images)

# Get predicted labels
labels = custom_df.columns.tolist()[2:]
true_classes = custom_df.iloc[:,2:].values==1
selected_predictions = predicted_vals[custom_df.index,:]>0.5

true_labels, pred_labels = [], []
for i in np.arange(0, selected_predictions.shape[0]):
    true_labels.append(list(compress(labels, true_classes[i,:])))
    pred_labels.append(list(compress(labels, selected_predictions[i,:])))

# Plot a processed image
sns.set_style("white")
plt.figure(figsize=(20,10))
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=1.5,
                    wspace=0.4,
                    hspace=0.4)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()
    plt.title('True class: ' + '\n' +  ', '.join(true_labels[i]) + \
              '\n' 'Predicted class: ' + '\n' + ', '.join(pred_labels[i]) + '\n', fontsize=16)

In [ ]:
# Visualize images for true class "Mass"
class_label = 'Mass'
test_class = test_df.loc[test_df[class_label]==1,:]

# Define number of images to plot
number_of_images = 9
custom_df = test_class.sample(n=number_of_images)

# Get predicted labels
labels = custom_df.columns.tolist()[2:]
true_classes = custom_df.iloc[:,2:].values==1
selected_predictions = predicted_vals[custom_df.index,:]>0.5

true_labels, pred_labels = [], []
for i in np.arange(0, selected_predictions.shape[0]):
    true_labels.append(list(compress(labels, true_classes[i,:])))
    pred_labels.append(list(compress(labels, selected_predictions[i,:])))

# Plot a processed image
sns.set_style("white")
plt.figure(figsize=(20,10))
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=1.5,
                    wspace=0.4,
                    hspace=0.4)

for i, image_name in enumerate(custom_df['Image'].values):
    plt.subplot(3, 3, i + 1)
    image = plt.imread(os.path.join("../nih/images_small/", image_name))
    plt.imshow(image, cmap='gray')
    plt.colorbar()
    plt.title('True class: ' + '\n' +  ', '.join(true_labels[i]) + \
              '\n' 'Predicted class: ' + '\n' + ', '.join(pred_labels[i]) + '\n', fontsize=16)